# Combining HealthLandscapes and COVID-19 case data sets
# Sidney Cannon-Bailey
# 10/24/2020

This notebook was created to begin stitching all the data sources together. The data that should be combined is:

- Covid-19 data, currently separated by city and organized by zip code
- Health Landscapes data, currently separated by city and organized by census tract
- Racial population data, all cities together, organized by zip code




In [1]:
# import libraries
import os
import pandas as pd

In [48]:
############## FUNCTIONS ##############

def convertZipToTract(dfToConvert, zipTractDf):
    
    # merge the two on the zip codes present in the covid df
    convertedDf = pd.merge(dfToConvert, zipTractDf, left_on='Zip Code', right_on = 'ZIP', how='left')

    return convertedDf



def loadAndMergeCovidHealthData(healthFileLoc, covidFileLoc, zipTractDf):
    
    # load the healh landscapes data and covid data as dataframes
    healthDf = pd.read_csv(healthFileLoc);
    covidDfTemp = pd.read_csv(covidFileLoc);
    
    # convert zip codes in covid datasets to zcta (census tracts)
    covidDf = convertZipToTract(covidDfTemp, zipTractDf);
    
    # merge the health landscapes and covid datasets using the census tract
    # only keep the zip codes referenced in the health landscapes df, since we're more sure that's the city limits
    fullDf = pd.merge(healthDf, covidDf, left_on='TractFIPS', right_on='TRACT', how='left');
    
    # remove unnecessary columns
    del fullDf['ZIP']
    del fullDf['TRACT']
    
    return fullDf



In [38]:
# set file location on your local machine
dataLoc = 'C:/Users/sidne/OneDrive/Documents/GeorgiaTech/ComputationalDataAnalytics_ISYE6740/Project/Datasets/';
healthLandFolder = dataLoc + 'healthLandscapesData/'
covidFolder = dataLoc + 'covid19Data/'
zipTractConvertLoc = dataLoc + 'zipTractConversion.csv'
racialDataLoc = dataLoc + 'nhgisRacialData.csv'

In [17]:
# import the zip <--> census tract conversion csv
zipTractDf = pd.read_csv(zipTractConvertLoc)

In [18]:
# Load each atlanta's info
atlantaDf = loadAndMergeCovidHealthData(healthLandFolder+'atlanta_healthLandscapes.csv', covidFolder+'fultonCounty_10_13.csv', zipTractDf);

atlantaDf.head()

,TractFIPS_1,PlaceName_1,Population2010,TractFIPS,BINGE_CrudePrev,CSMOKING_CrudePrev,LPA_CrudePrev,OBESITY_CrudePrev,SLEEP_CrudePrev,ARTHRITIS_CrudePrev,...,DENTAL_CrudePrev,BPMED_CrudePrev,CHOLSCREEN_CrudePrev,MAMMOUSE_CrudePrev,PAPTEST_CrudePrev,COLON_SCREEN_CrudePrev,COREM_CrudePrev,COREW_CrudePrev,Zip Code,Confirmed Cases
0,13089020100,"Atlanta, GA",1492,13089020100,19.6,9.6,18,21.5,29.8,16.1,...,84.8,70.8,87.3,80.2,87,74.9,45.0,42.5,30306.0,353
1,13089020200,"Atlanta, GA",1943,13089020200,21.2,10.4,18,20.8,30.8,14.7,...,81.9,69.5,83.7,79.9,87,74.4,44.1,42.3,30306.0,353
2,13089020200,"Atlanta, GA",1943,13089020200,21.2,10.4,18,20.8,30.8,14.7,...,81.9,69.5,83.7,79.9,87,74.4,44.1,42.3,30307.0,203
3,13089020300,"Atlanta, GA",3574,13089020300,21.0,11.3,18,22.4,31.7,14.3,...,82.7,66.4,85.4,80.4,88,72.3,42.1,40.7,30307.0,203
4,13089020400,"Atlanta, GA",2376,13089020400,22.7,10.3,17,21.0,30.9,12.0,...,84.2,62.3,83.6,80.5,88,74.2,43.4,42.5,30307.0,203


Looks like there are multiple census tracts per zip code. Looking this up in more detail, it appears that census tracts are not 1:1 with zip codes.

I think it's a fair assumption that zip code is a sufficiently small division of location and that someone in the same zip code has a pretty similar health metrics. Therefore, moving forward I will use the zipTractDf to find census tracts that exist in the same zip code and average all the health landscapes metrics over the census tracts in this zip code.

The setup to do this will be significantly different, so I will branch off to the next version (v2) to do this

In [19]:

bostonDf = loadAndMergeCovidHealthData(healthLandFolder+'boston_healthLandscapes.csv', covidFolder+'boston_10_14.csv', zipTractDf);
bostonDf.head()

#### NOTE: The Boston dataset had multiple zip codes reported in one category
# To change this, I divided the number positive cases into each zip code, weighting by each zip code's population



,TractFIPS_1,PlaceName_1,Population2010,TractFIPS,BINGE_CrudePrev,CSMOKING_CrudePrev,LPA_CrudePrev,OBESITY_CrudePrev,SLEEP_CrudePrev,ARTHRITIS_CrudePrev,...,DENTAL_CrudePrev,BPMED_CrudePrev,CHOLSCREEN_CrudePrev,MAMMOUSE_CrudePrev,PAPTEST_CrudePrev,COLON_SCREEN_CrudePrev,COREM_CrudePrev,COREW_CrudePrev,Zip Code,Confirmed Cases
0,25025000100,"Boston, MA",4254,25025000100,25.9,17.2,22,21.8,34.5,14.3,...,66.7,64.3,71.7,81.3,84,63.4,29.0,29.0,2163.0,1302.0
1,25025000201,"Boston, MA",3854,25025000201,28.7,13.2,17,19.2,31.8,12.1,...,75.6,61.7,73.6,84.4,86,68.5,31.7,31.8,NaN,NaN
2,25025000202,"Boston, MA",3885,25025000202,24.8,15.8,21,21.7,33.9,15.3,...,69.5,67.4,74.0,82.8,85,63.6,29.1,29.8,NaN,NaN
3,25025000301,"Boston, MA",2740,25025000301,26.8,13.0,18,19.1,32.1,14.9,...,74.5,68.2,76.6,83.0,85,68.7,32.1,32.0,NaN,NaN
4,25025000302,"Boston, MA",3142,25025000302,26.8,12.0,17,17.9,31.4,14.1,...,74.7,67.6,75.5,83.4,84,69.5,31.6,31.9,NaN,NaN


In [20]:
chicagoDf = loadAndMergeCovidHealthData(healthLandFolder+'chicago_healthLandscapes.csv', covidFolder+'illinois_10_15.csv', zipTractDf);
chicagoDf.head()



,TractFIPS_1,PlaceName_1,Population2010,TractFIPS,BINGE_CrudePrev,CSMOKING_CrudePrev,LPA_CrudePrev,OBESITY_CrudePrev,SLEEP_CrudePrev,ARTHRITIS_CrudePrev,...,DENTAL_CrudePrev,BPMED_CrudePrev,CHOLSCREEN_CrudePrev,MAMMOUSE_CrudePrev,PAPTEST_CrudePrev,COLON_SCREEN_CrudePrev,COREM_CrudePrev,COREW_CrudePrev,Zip Code,Confirmed Cases
0,17031010100,"Chicago, IL",4854,17031010100,20.6,20.3,28,36.4,39.0,20.3,...,54.7,70.3,82.2,79.8,87,54.5,29.4,23.8,60626,"1,516"
1,17031010201,"Chicago, IL",6450,17031010201,21.0,22.5,30,36.2,39.0,18.9,...,50.3,67.5,79.6,78.6,86,50.5,28.0,22.1,60626,"1,516"
2,17031010201,"Chicago, IL",6450,17031010201,21.0,22.5,30,36.2,39.0,18.9,...,50.3,67.5,79.6,78.6,86,50.5,28.0,22.1,60645,"1,549"
3,17031010202,"Chicago, IL",2818,17031010202,21.0,20.1,28,33.9,36.8,22.0,...,55.6,72.4,82.5,78.4,86,56.5,31.2,24.2,60626,"1,516"
4,17031010300,"Chicago, IL",6236,17031010300,21.0,19.0,27,31.8,35.1,22.8,...,59.7,73.3,83.0,77.0,86,59.3,33.6,25.2,60626,"1,516"


In [21]:
houstonDf = loadAndMergeCovidHealthData(healthLandFolder+'houston_healthLandscapes.csv', covidFolder+'harrisCounty_10_14.csv', zipTractDf);
houstonDf.head()



,TractFIPS_1,PlaceName_1,Population2010,TractFIPS,BINGE_CrudePrev,CSMOKING_CrudePrev,LPA_CrudePrev,OBESITY_CrudePrev,SLEEP_CrudePrev,ARTHRITIS_CrudePrev,...,DENTAL_CrudePrev,BPMED_CrudePrev,CHOLSCREEN_CrudePrev,MAMMOUSE_CrudePrev,PAPTEST_CrudePrev,COLON_SCREEN_CrudePrev,COREM_CrudePrev,COREW_CrudePrev,Zip Code,Confirmed Cases
0,48157670101,"Houston, TX",6774,48157670101,15.2,23.0,43,42.5,40.1,20.9,...,42.4,71.3,74.3,79.2,81,49.7,21.0,21.7,77053,739
1,48157670102,"Houston, TX",3383,48157670102,15.5,19.8,37,42.1,40.7,21.3,...,53.7,74.1,79.9,81.6,83,58.7,25.0,27.1,77053,739
2,48157670200,"Houston, TX",8322,48157670200,15.9,19.4,36,41.1,40.6,21.5,...,50.4,74.3,79.8,81.7,83,55.9,24.8,25.3,77053,739
3,48157670200,"Houston, TX",8322,48157670200,15.9,19.4,36,41.1,40.6,21.5,...,50.4,74.3,79.8,81.7,83,55.9,24.8,25.3,77085,833
4,48157670200,"Houston, TX",8322,48157670200,15.9,19.4,36,41.1,40.6,21.5,...,50.4,74.3,79.8,81.7,83,55.9,24.8,25.3,77489,233


In [35]:
miamiDf = loadAndMergeCovidHealthData(healthLandFolder+'miami_healthLandscapes.csv', covidFolder+'dadeCounty_10_15.csv', zipTractDf);
miamiDf.head()



,TractFIPS_1,PlaceName_1,Population2010,TractFIPS,BINGE_CrudePrev,CSMOKING_CrudePrev,LPA_CrudePrev,OBESITY_CrudePrev,SLEEP_CrudePrev,ARTHRITIS_CrudePrev,...,DENTAL_CrudePrev,BPMED_CrudePrev,CHOLSCREEN_CrudePrev,MAMMOUSE_CrudePrev,PAPTEST_CrudePrev,COLON_SCREEN_CrudePrev,COREM_CrudePrev,COREW_CrudePrev,Zip Code,Confirmed Cases
0,12086001002,"Miami, FL",364,12086001002,16.1,23.0,36,34.7,40.6,15.7,...,50.7,66.1,75.0,85.3,82,48.1,18.2,17.9,33147,3459
1,12086001002,"Miami, FL",364,12086001002,16.1,23.0,36,34.7,40.6,15.7,...,50.7,66.1,75.0,85.3,82,48.1,18.2,17.9,33150,1620
2,12086001004,"Miami, FL",347,12086001004,12.9,30.6,44,42.6,45.2,21.3,...,38.9,72.9,75.1,86.6,81,39.8,14.7,15.5,33147,3459
3,12086001004,"Miami, FL",347,12086001004,12.9,30.6,44,42.6,45.2,21.3,...,38.9,72.9,75.1,86.6,81,39.8,14.7,15.5,33150,1620
4,12086001301,"Miami, FL",4544,12086001301,13.8,24.7,39,37.3,38.8,22.6,...,49.6,75.4,78.9,84.2,81,46.6,20.3,18.6,33138,1037


In [33]:
sanFranciscoDf = loadAndMergeCovidHealthData(healthLandFolder+'sanfrancisco_healthLandscapes.csv', covidFolder+'sanFrancisco_10_14.csv', zipTractDf);
sanFranciscoDf.head()

,TractFIPS_1,PlaceName_1,Population2010,TractFIPS,BINGE_CrudePrev,CSMOKING_CrudePrev,LPA_CrudePrev,OBESITY_CrudePrev,SLEEP_CrudePrev,ARTHRITIS_CrudePrev,...,DENTAL_CrudePrev,BPMED_CrudePrev,CHOLSCREEN_CrudePrev,MAMMOUSE_CrudePrev,PAPTEST_CrudePrev,COLON_SCREEN_CrudePrev,COREM_CrudePrev,COREW_CrudePrev,Zip Code,Confirmed Cases
0,6075010100,"San Francisco, CA",3739,6075010100,19.8,11.5,17,16.7,30.2,17.2,...,67.9,71.0,80.7,79.2,83,67.4,32.7,29.0,94133,153
1,6075010100,"San Francisco, CA",3739,6075010100,19.8,11.5,17,16.7,30.2,17.2,...,67.9,71.0,80.7,79.2,83,67.4,32.7,29.0,94111,16
2,6075010200,"San Francisco, CA",4143,6075010200,25.2,8.3,11,16.9,25.7,16.0,...,80.4,66.9,82.9,82.2,88,77.4,40.9,36.9,94133,153
3,6075010200,"San Francisco, CA",4143,6075010200,25.2,8.3,11,16.9,25.7,16.0,...,80.4,66.9,82.9,82.2,88,77.4,40.9,36.9,94109,677
4,6075010300,"San Francisco, CA",3852,6075010300,22.4,10.2,14,15.8,28.3,16.4,...,73.6,69.6,81.6,80.0,83,69.6,35.0,31.6,94133,153


In [36]:
# combine them into one dataframe
allDf = pd.concat([atlantaDf, bostonDf, chicagoDf, houstonDf, miamiDf, sanFranciscoDf])


In [37]:
allDf.head()

,TractFIPS_1,PlaceName_1,Population2010,TractFIPS,BINGE_CrudePrev,CSMOKING_CrudePrev,LPA_CrudePrev,OBESITY_CrudePrev,SLEEP_CrudePrev,ARTHRITIS_CrudePrev,...,DENTAL_CrudePrev,BPMED_CrudePrev,CHOLSCREEN_CrudePrev,MAMMOUSE_CrudePrev,PAPTEST_CrudePrev,COLON_SCREEN_CrudePrev,COREM_CrudePrev,COREW_CrudePrev,Zip Code,Confirmed Cases
0,13089020100,"Atlanta, GA",1492,13089020100,19.6,9.6,18,21.5,29.8,16.1,...,84.8,70.8,87.3,80.2,87,74.9,45.0,42.5,30306.0,353
1,13089020200,"Atlanta, GA",1943,13089020200,21.2,10.4,18,20.8,30.8,14.7,...,81.9,69.5,83.7,79.9,87,74.4,44.1,42.3,30306.0,353
2,13089020200,"Atlanta, GA",1943,13089020200,21.2,10.4,18,20.8,30.8,14.7,...,81.9,69.5,83.7,79.9,87,74.4,44.1,42.3,30307.0,203
3,13089020300,"Atlanta, GA",3574,13089020300,21.0,11.3,18,22.4,31.7,14.3,...,82.7,66.4,85.4,80.4,88,72.3,42.1,40.7,30307.0,203
4,13089020400,"Atlanta, GA",2376,13089020400,22.7,10.3,17,21.0,30.9,12.0,...,84.2,62.3,83.6,80.5,88,74.2,43.4,42.5,30307.0,203


In [43]:
# load race by zip code
racialDf = pd.read_csv(racialDataLoc)

# convert from zip code to census tract
#racialDf = convertZipToTract(racialDf, zipTractDf);

# add racial data to overall data frame, keeping only the zip codes in the overall df
finalDf = pd.merge(allDf, racialDf, left_on='Zip Code', right_on = 'Zip Code', how='left')




In [47]:
finalDf.head()


,TractFIPS_1,PlaceName_1,Population2010,TractFIPS,BINGE_CrudePrev,CSMOKING_CrudePrev,LPA_CrudePrev,OBESITY_CrudePrev,SLEEP_CrudePrev,ARTHRITIS_CrudePrev,...,Zip Code,Confirmed Cases,Total,whiteAlone,blackAlone,nativeAlone,asianAlone,hawaiiAlone,otherAlone,twoOrMore
0,13089020100,"Atlanta, GA",1492,13089020100,19.6,9.6,18,21.5,29.8,16.1,...,30306.0,353,22246.0,19345.0,1359.0,59.0,726.0,7.0,318.0,432.0
1,13089020200,"Atlanta, GA",1943,13089020200,21.2,10.4,18,20.8,30.8,14.7,...,30306.0,353,22246.0,19345.0,1359.0,59.0,726.0,7.0,318.0,432.0
2,13089020200,"Atlanta, GA",1943,13089020200,21.2,10.4,18,20.8,30.8,14.7,...,30307.0,203,18004.0,13941.0,2803.0,33.0,561.0,3.0,193.0,470.0
3,13089020300,"Atlanta, GA",3574,13089020300,21.0,11.3,18,22.4,31.7,14.3,...,30307.0,203,18004.0,13941.0,2803.0,33.0,561.0,3.0,193.0,470.0
4,13089020400,"Atlanta, GA",2376,13089020400,22.7,10.3,17,21.0,30.9,12.0,...,30307.0,203,18004.0,13941.0,2803.0,33.0,561.0,3.0,193.0,470.0
